# Lab-05 YOLOv7

Please run the code with "VScode-devcontainer".

> You can find the tutorial provided by Visual Studio Code here :   
> [https://code.visualstudio.com/docs/devcontainers/containers](https://code.visualstudio.com/docs/devcontainers/containers)

## Import Required Libraries

In [15]:
from argparse import Namespace
from copy import deepcopy
from pathlib import Path

import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from tqdm import tqdm

import test  # import test.py to get mAP after each epoch
from models.yolo import Model
from utils.autoanchor import check_anchors
from utils.datasets import create_dataloader
from utils.general import labels_to_class_weights, increment_path, init_seeds, fitness, check_dataset, check_file, check_img_size, one_cycle, colorstr
from utils.loss import ComputeLoss, ComputeLossOTA
from utils.torch_utils import ModelEMA, intersect_dicts

## Hyper-Parameters

In [16]:
OPTIMIZER_NAME = "SGD"
BATCH_SIZE = 4
EPOCH = 2

## utils

In [17]:
# def make_divisible(x, divisor):
#     # Returns x evenly divisible by divisor
#     return math.ceil(x / divisor) * divisor

## Get Device

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Prepare Dataset for fine-tune

- Dataset : Aquarium Dataset
    - [https://public.roboflow.com/object-detection/aquarium](https://public.roboflow.com/object-detection/aquarium)

---

- Dataloader
    - [pin_memory](https://pytorch.org/docs/stable/data.html#memory-pinning)
    - [Train Custom Data (From YOLOv5)](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data)

---

wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [19]:
# def create_dataloader(path, imgsz, batch_size, stride, opt, hyp=None, augment=False, cache=False, pad=0.0, rect=False, rank=-1, world_size=1, workers=8, image_weights=False, quad=False, prefix=""):

#     dataset = LoadImagesAndLabels(path, imgsz, batch_size, augment=augment, hyp=hyp, rect=rect, cache_images=cache, single_cls=opt.single_cls, stride=int(stride), pad=pad, image_weights=image_weights, prefix=prefix)  # augment images  # augmentation hyperparameters  # rectangular training

#     batch_size = min(batch_size, len(dataset))
#     nw = min([os.cpu_count() // world_size, batch_size if batch_size > 1 else 0, workers])  # number of workers
#     sampler = torch.utils.data.distributed.DistributedSampler(dataset) if rank != -1 else None
#     loader = torch.utils.data.DataLoader if image_weights else InfiniteDataLoader
#     # Use torch.utils.data.DataLoader() if dataset.properties will update during training else InfiniteDataLoader()
#     dataloader = loader(dataset, batch_size=batch_size, num_workers=nw, sampler=sampler, pin_memory=True, collate_fn=LoadImagesAndLabels.collate_fn4 if quad else LoadImagesAndLabels.collate_fn)
#     return dataloader


# # Trainloader
# dataloader = create_dataloader(path=data_dict["train"], imgsz=imgsz, batch_size=BATCH_SIZE, stride=32, opt=opt, hyp=hyp, augment=True, prefix=colorstr("train: "))
# testloader = create_dataloader(path=data_dict["val"], imgsz=imgsz_test, batch_size=BATCH_SIZE * 2, stride=32, opt=opt, hyp=hyp, rect=True, pad=0.5, prefix=colorstr("val: "))[0]  # testloader

## Download pre-trained weight

Attempt to download the file if it does not exist.  

- Reference  
    - [https://github.com/WongKinYiu/yolov7/releases/tag/v0.1](https://github.com/WongKinYiu/yolov7/releases/tag/v0.1)

In [20]:
def attempt_download(filename):
    file = Path(filename)

    # download if not found locally
    if not file.exists():
        repo = "WongKinYiu/yolov7"
        tag = "v0.1"
        name = file.name
        url = f"https://github.com/{repo}/releases/download/{tag}/{name}"
        print(f"Downloading {url} to {file}...")
        torch.hub.download_url_to_file(url, file)

attempt_download(filename="yolov7_training.pt")  

In [21]:
def train(hyp, opt, device):
    print(colorstr("hyperparameters: ") + ", ".join(f"{k}={v}" for k, v in hyp.items()))
    save_dir, epochs, batch_size, weights = Path(opt.save_dir), opt.epochs, opt.batch_size, opt.weights

    # Directories
    wdir = save_dir / "weights"
    wdir.mkdir(parents=True, exist_ok=True)  # make dir
    best = wdir / "best.pt"
    results_file = save_dir / "results.txt"

    # Save run settings
    with open(save_dir / "hyp.yaml", "w") as f:
        yaml.dump(hyp, f, sort_keys=False)
    with open(save_dir / "opt.yaml", "w") as f:
        yaml.dump(vars(opt), f, sort_keys=False)

    # Configure
    plots = True  # create plots
    cuda = device.type != "cpu"
    init_seeds(1)
    with open(opt.data) as f:
        data_dict = yaml.load(f, Loader=yaml.SafeLoader)  # data dict

    # Logging- Doing this before checking the dataset. Might update data_dict
    opt.hyp = hyp  # add hyperparameters

    nc = int(data_dict["nc"])  # number of classes
    names = data_dict["names"]  # class names

    # Model
    attempt_download(weights)  # download if not found locally
    ckpt = torch.load(weights, map_location=device)  # load checkpoint
    model = Model(opt.cfg or ckpt["model"].yaml, ch=3, nc=nc, anchors=hyp.get("anchors")).to(device)  # create
    state_dict = ckpt["model"].float().state_dict()  # to FP32
    state_dict = intersect_dicts(state_dict, model.state_dict(), exclude=["anchor"])  # intersect
    model.load_state_dict(state_dict, strict=False)  # load
    print("Transferred %g/%g items from %s" % (len(state_dict), len(model.state_dict()), weights))  # report

    check_dataset(data_dict)  # check

    # Freeze
    freeze = [f"model.{x}." for x in range(0)]  # parameter names to freeze (full or partial)
    for k, v in model.named_parameters():
        v.requires_grad = True  # train all layers
        if any(x in k for x in freeze):
            print("freezing %s" % k)
            v.requires_grad = False

    # Optimizer
    nbs = 64  # nominal batch size
    accumulate = max(round(nbs / batch_size), 1)  # accumulate loss before optimizing
    hyp["weight_decay"] *= batch_size * accumulate / nbs  # scale weight_decay
    print(f"Scaled weight_decay = {hyp['weight_decay']}")

    pg0, pg1, pg2 = [], [], []  # optimizer parameter groups
    for k, v in model.named_modules():
        if hasattr(v, "bias") and isinstance(v.bias, nn.Parameter):
            pg2.append(v.bias)  # biases
        if isinstance(v, nn.BatchNorm2d):
            pg0.append(v.weight)  # no decay
        elif hasattr(v, "weight") and isinstance(v.weight, nn.Parameter):
            pg1.append(v.weight)  # apply decay
        if hasattr(v, "im"):
            if hasattr(v.im, "implicit"):
                pg0.append(v.im.implicit)
            else:
                for iv in v.im:
                    pg0.append(iv.implicit)
        if hasattr(v, "imc"):
            if hasattr(v.imc, "implicit"):
                pg0.append(v.imc.implicit)
            else:
                for iv in v.imc:
                    pg0.append(iv.implicit)
        if hasattr(v, "imb"):
            if hasattr(v.imb, "implicit"):
                pg0.append(v.imb.implicit)
            else:
                for iv in v.imb:
                    pg0.append(iv.implicit)
        if hasattr(v, "imo"):
            if hasattr(v.imo, "implicit"):
                pg0.append(v.imo.implicit)
            else:
                for iv in v.imo:
                    pg0.append(iv.implicit)
        if hasattr(v, "ia"):
            if hasattr(v.ia, "implicit"):
                pg0.append(v.ia.implicit)
            else:
                for iv in v.ia:
                    pg0.append(iv.implicit)
        if hasattr(v, "attn"):
            if hasattr(v.attn, "logit_scale"):
                pg0.append(v.attn.logit_scale)
            if hasattr(v.attn, "q_bias"):
                pg0.append(v.attn.q_bias)
            if hasattr(v.attn, "v_bias"):
                pg0.append(v.attn.v_bias)
            if hasattr(v.attn, "relative_position_bias_table"):
                pg0.append(v.attn.relative_position_bias_table)
        if hasattr(v, "rbr_dense"):
            if hasattr(v.rbr_dense, "weight_rbr_origin"):
                pg0.append(v.rbr_dense.weight_rbr_origin)
            if hasattr(v.rbr_dense, "weight_rbr_avg_conv"):
                pg0.append(v.rbr_dense.weight_rbr_avg_conv)
            if hasattr(v.rbr_dense, "weight_rbr_pfir_conv"):
                pg0.append(v.rbr_dense.weight_rbr_pfir_conv)
            if hasattr(v.rbr_dense, "weight_rbr_1x1_kxk_idconv1"):
                pg0.append(v.rbr_dense.weight_rbr_1x1_kxk_idconv1)
            if hasattr(v.rbr_dense, "weight_rbr_1x1_kxk_conv2"):
                pg0.append(v.rbr_dense.weight_rbr_1x1_kxk_conv2)
            if hasattr(v.rbr_dense, "weight_rbr_gconv_dw"):
                pg0.append(v.rbr_dense.weight_rbr_gconv_dw)
            if hasattr(v.rbr_dense, "weight_rbr_gconv_pw"):
                pg0.append(v.rbr_dense.weight_rbr_gconv_pw)
            if hasattr(v.rbr_dense, "vector"):
                pg0.append(v.rbr_dense.vector)

    if OPTIMIZER_NAME == "Adam":
        optimizer = optim.Adam(pg0, lr=hyp["lr0"], betas=(hyp["momentum"], 0.999))  # adjust beta1 to momentum
    elif OPTIMIZER_NAME == "SGD":
        optimizer = optim.SGD(pg0, lr=hyp["lr0"], momentum=hyp["momentum"], nesterov=True)

    optimizer.add_param_group({"params": pg1, "weight_decay": hyp["weight_decay"]})  # add pg1 with weight_decay
    optimizer.add_param_group({"params": pg2})  # add pg2 (biases)
    print("Optimizer groups: %g .bias, %g conv.weight, %g other" % (len(pg2), len(pg1), len(pg0)))
    del pg0, pg1, pg2

    lf = one_cycle(1, hyp["lrf"], epochs)  # cosine 1->hyp['lrf']
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)

    # EMA
    ema = ModelEMA(model)

    # Resume
    start_epoch, best_fitness = 0, 0.0

    # Optimizer
    if ckpt["optimizer"] is not None:
        optimizer.load_state_dict(ckpt["optimizer"])
        best_fitness = ckpt["best_fitness"]

    # EMA
    if ckpt.get("ema"):
        ema.ema.load_state_dict(ckpt["ema"].float().state_dict())
        ema.updates = ckpt["updates"]

    # Results
    if ckpt.get("training_results") is not None:
        results_file.write_text(ckpt["training_results"])  # write results.txt

    # Epochs
    start_epoch = ckpt["epoch"] + 1
    if epochs < start_epoch:
        print("%s has been trained for %g epochs. Fine-tuning for %g additional epochs." % (weights, ckpt["epoch"], epochs))
        epochs += ckpt["epoch"]  # finetune additional epochs
    del ckpt, state_dict

    # Image sizes
    nl = model.model[-1].nl  # number of detection layers (used for scaling hyp['obj'])
    imgsz, imgsz_test = [check_img_size(x, 32) for x in opt.img_size]  # verify imgsz are gs-multiples

    # Trainloader
    dataloader, dataset = create_dataloader(path=data_dict["train"], imgsz=imgsz, batch_size=batch_size, stride=32, opt=opt, hyp=hyp, augment=True, prefix=colorstr("train: "))
    testloader = create_dataloader(path=data_dict["val"], imgsz=imgsz_test, batch_size=batch_size * 2, stride=32, opt=opt, hyp=hyp, rect=True, pad=0.5, prefix=colorstr("val: "))[0]  # testloader
    nb = len(dataloader)  # number of batches

    labels = np.concatenate(dataset.labels, 0)
    c = torch.tensor(labels[:, 0])  # classes

    # Anchors
    if not opt.noautoanchor:
        check_anchors(dataset, model=model, thr=hyp["anchor_t"], imgsz=imgsz)
    model.half().float()  # pre-reduce anchor precision

    # Model parameters
    hyp["box"] *= 3.0 / nl  # scale to layers
    hyp["cls"] *= nc / 80.0 * 3.0 / nl  # scale to classes and layers
    hyp["obj"] *= (imgsz / 640) ** 2 * 3.0 / nl  # scale to image size and layers
    hyp["label_smoothing"] = 0.0
    model.nc = nc  # attach number of classes to model
    model.hyp = hyp  # attach hyperparameters to model
    model.gr = 1.0  # iou loss ratio (obj_loss = 1.0 or iou)
    model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device) * nc  # attach class weights
    model.names = names

    # Start training
    nw = max(round(hyp["warmup_epochs"] * nb), 1000)  # number of warmup iterations, max(3 epochs, 1k iterations)
    results = (0, 0, 0, 0, 0, 0, 0)  # P, R, mAP@.5, mAP@.5-.95, val_loss(box, obj, cls)
    scheduler.last_epoch = start_epoch - 1  # do not move
    scaler = amp.GradScaler(enabled=cuda)
    compute_loss_ota = ComputeLossOTA(model)  # init loss class
    compute_loss = ComputeLoss(model)  # init loss class
    print(f"Image sizes {imgsz} train, {imgsz_test} test\n" f"Using {dataloader.num_workers} dataloader workers\n" f"Logging results to {save_dir}\n" f"Starting training for {epochs} epochs...")
    
    for epoch in range(start_epoch, epochs):
        model.train()

        mloss = torch.zeros(4, device=device)  # mean losses
        print(("\n" + "%10s" * 8) % ("Epoch", "gpu_mem", "box", "obj", "cls", "total", "labels", "img_size"))
        pbar = tqdm(enumerate(dataloader), total=nb)  # progress bar
        optimizer.zero_grad()

        # batch -------------------------------------------------------------
        for i, (imgs, targets, paths, _) in pbar:
            ni = i + nb * epoch  # number integrated batches (since train start)
            imgs = imgs.to(device, non_blocking=True).float() / 255.0  # uint8 to float32, 0-255 to 0.0-1.0

            # Warmup
            if ni <= nw:
                xi = [0, nw]  # x interp
                # model.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
                accumulate = max(1, np.interp(ni, xi, [1, nbs / batch_size]).round())
                for j, x in enumerate(optimizer.param_groups):
                    # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                    x["lr"] = np.interp(ni, xi, [hyp["warmup_bias_lr"] if j == 2 else 0.0, x["initial_lr"] * lf(epoch)])
                    if "momentum" in x:
                        x["momentum"] = np.interp(ni, xi, [hyp["warmup_momentum"], hyp["momentum"]])

            # Forward
            with amp.autocast(enabled=cuda):
                pred = model(imgs)  # forward
                if "loss_ota" not in hyp or hyp["loss_ota"] == 1:
                    loss, loss_items = compute_loss_ota(pred, targets.to(device), imgs)  # loss scaled by batch_size
                else:
                    loss, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size

            # Backward
            scaler.scale(loss).backward()

            # Optimize
            if ni % accumulate == 0:
                scaler.step(optimizer)  # optimizer.step
                scaler.update()
                optimizer.zero_grad()
                if ema:
                    ema.update(model)

            # Print
            mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
            mem = "%.3gG" % (torch.cuda.memory_reserved() / 1e9 if torch.cuda.is_available() else 0)  # (GB)
            s = ("%10s" * 2 + "%10.4g" * 6) % ("%g/%g" % (epoch, epochs - 1), mem, *mloss, targets.shape[0], imgs.shape[-1])
            pbar.set_description(s)

        # Scheduler
        scheduler.step()

        # mAP
        ema.update_attr(model, include=["yaml", "nc", "hyp", "gr", "names", "stride", "class_weights"])
        final_epoch = epoch + 1 == epochs
        results, maps, times = test.test(data_dict, batch_size=batch_size * 2, imgsz=imgsz_test, model=ema.ema, dataloader=testloader, save_dir=save_dir, verbose=nc < 50 and final_epoch, plots=plots and final_epoch, compute_loss=compute_loss)

        # Write
        with open(results_file, "a") as f:
            f.write(s + "%10.4g" * 7 % results + "\n")  # append metrics, val_loss

        # Update best mAP
        fi = fitness(np.array(results).reshape(1, -1))  # weighted combination of [P, R, mAP@.5, mAP@.5-.95]
        if fi > best_fitness:
            best_fitness = fi

        # Save model
        if final_epoch:  # if save
            ckpt = {"epoch": epoch, "best_fitness": best_fitness, "training_results": results_file.read_text(), "model": deepcopy(model).half(), "ema": deepcopy(ema.ema).half(), "updates": ema.updates, "optimizer": optimizer.state_dict(), "wandb_id": None}
            torch.save(ckpt, best)

    torch.cuda.empty_cache()
    return results

---

In [22]:
opt = Namespace()

opt.weights = "yolov7_training.pt"
opt.noautoanchor = False
opt.single_cls = False

opt.batch_size = BATCH_SIZE
opt.epochs = EPOCH

opt.data, opt.cfg, opt.hyp = check_file("data/AquariumDataset/data.yaml"), check_file("cfg/training/yolov7.yaml"), check_file("data/hyp.scratch.custom.yaml")  # check files
opt.img_size = [640, 640]
opt.save_dir = increment_path(Path("runs/train") / "yolov7-custom", exist_ok=False)  # increment run

# Hyperparameters
with open(opt.hyp) as f:
    hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps

# Train
print(opt)
train(hyp, opt, device)

Namespace(weights='yolov7_training.pt', noautoanchor=False, single_cls=False, batch_size=4, epochs=2, data='data/AquariumDataset/data.yaml', cfg='cfg/training/yolov7.yaml', hyp='data/hyp.scratch.custom.yaml', img_size=[640, 640], save_dir='runs/train/yolov7-custom2')
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.2, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0, paste_in=0.0, loss_ota=1


Transferred 555/566 items from yolov7_training.pt
Scaled weight_decay = 0.0005
Optimizer groups: 95 .bias, 95 conv.weight, 98 other


train: Scanning 'data/AquariumDataset/train/labels.cache' images and labels... 448 found, 0 missing, 1 empty, 0 corrupted: 100%|██████████| 448/448 [00:00<?, ?it/s]
val: Scanning 'data/AquariumDataset/valid/labels.cache' images and labels... 127 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 127/127 [00:00<?, ?it/s]



autoanchor: Analyzing anchors... anchors/target = 5.03, Best Possible Recall (BPR) = 0.9991
Image sizes 640 train, 640 test
Using 4 dataloader workers
Logging results to runs/train/yolov7-custom2
Starting training for 2 epochs...

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size


       0/1      3.4G   0.07729   0.02407   0.03187    0.1332        29       640: 100%|██████████| 112/112 [00:14<00:00,  7.85it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 16/16 [00:01<00:00, 14.36it/s]


                 all         127         909     0.00797      0.0417     0.00152    0.000292

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size


       1/1      4.6G   0.06619   0.02692   0.02818    0.1213        56       640: 100%|██████████| 112/112 [00:13<00:00,  8.10it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 16/16 [00:01<00:00, 12.36it/s]


                 all         127         909      0.0374       0.194      0.0214     0.00574
                fish         127         459      0.0551       0.458      0.0797      0.0184
           jellyfish         127         155      0.0441       0.161      0.0158     0.00344
             penguin         127         104       0.042      0.0769     0.00785     0.00175
              puffin         127          74      0.0387       0.108     0.00575     0.00173
               shark         127          57      0.0112      0.0702     0.00539     0.00156
            starfish         127          27       0.051       0.333      0.0251     0.00979
            stingray         127          33      0.0196       0.152     0.00987     0.00351


(0.03737289769822971,
 0.1941231101322982,
 0.021350714084934944,
 0.005740058229584408,
 0.09389394521713257,
 0.04816664382815361,
 0.03097183257341385)